In [22]:
import pandas as pd
from warnings import simplefilter
import psycopg2
import psycopg2.extras

irs_data = pd.read_csv('./IRS_Data_2021.csv', header=0)

# filter by New York state
ny_df = irs_data[irs_data['STATE'] == 'NY'].reset_index(drop=True)



In [ ]:
# irs_data = pd.read_csv('./IRS_Data_2021.csv', header=0, dtype={'zipcode': int64})

In [23]:
zips = pd.read_csv('./nyc-zip-codes.csv', header=0)

zips.head()

,Borough,Neighborhood,ZipCode
0,Bronx,Central Bronx,10453
1,Bronx,Central Bronx,10457
2,Bronx,Central Bronx,10460
3,Bronx,Bronx Park and Fordham,10458
4,Bronx,Bronx Park and Fordham,10467


In [24]:
zips = zips.rename(columns={'ZipCode': 'zipcode'})
# df.rename(columns={"A": "a", "B": "c"})
zips.head()

,Borough,Neighborhood,zipcode
0,Bronx,Central Bronx,10453
1,Bronx,Central Bronx,10457
2,Bronx,Central Bronx,10460
3,Bronx,Bronx Park and Fordham,10458
4,Bronx,Bronx Park and Fordham,10467


In [25]:
ny_df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,36,NY,0,1,2691220.0,2004510.0,257970.0,375610.0,2526010.0,84410.0,...,0.0,0.0,333540.0,282862.0,2120650.0,5354373.0,2106320.0,5299094.0,15870.0,39469.0
1,36,NY,0,2,2324490.0,1357130.0,393190.0,515930.0,2242480.0,43080.0,...,0.0,0.0,421060.0,619115.0,1891640.0,5311653.0,1877170.0,5245116.0,19460.0,58182.0
2,36,NY,0,3,1424670.0,779250.0,348060.0,247450.0,1371930.0,25640.0,...,0.0,0.0,358700.0,731938.0,1065060.0,2850715.0,1049050.0,2766543.0,21670.0,78743.0
3,36,NY,0,4,908050.0,408800.0,342880.0,121210.0,871550.0,13540.0,...,0.0,0.0,285340.0,785409.0,617770.0,1944049.0,602360.0,1854707.0,16390.0,75357.0
4,36,NY,0,5,1493450.0,449260.0,876140.0,120400.0,1437630.0,24540.0,...,9530.0,2643.0,623100.0,2893434.0,869030.0,3678321.0,822860.0,3277872.0,57010.0,402093.0


In [11]:
ny_df = ny_df['zipcode'].astype(int)

In [26]:
nyc_irs = pd.merge(ny_df, zips, on='zipcode')

In [31]:
nyc_irs.head()


,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,CPREP,...,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000,Borough,Neighborhood
0,36,NY,10001,1,2960.0,2370.0,210.0,290.0,2770.0,90.0,...,420.0,426.0,2150.0,4855.0,2100.0,4666.0,50.0,159.0,Manhattan,Chelsea and Clinton
1,36,NY,10001,2,2470.0,1790.0,200.0,410.0,2370.0,60.0,...,580.0,1189.0,1860.0,5102.0,1820.0,4913.0,50.0,189.0,Manhattan,Chelsea and Clinton
2,36,NY,10001,3,1930.0,1530.0,180.0,160.0,1850.0,30.0,...,520.0,1376.0,1390.0,4045.0,1350.0,3859.0,50.0,186.0,Manhattan,Chelsea and Clinton
3,36,NY,10001,4,1560.0,1260.0,170.0,90.0,1500.0,20.0,...,540.0,1676.0,1000.0,2489.0,960.0,2114.0,50.0,375.0,Manhattan,Chelsea and Clinton
4,36,NY,10001,5,3270.0,2520.0,510.0,100.0,3140.0,50.0,...,1470.0,6680.0,1780.0,6655.0,1640.0,5050.0,170.0,1605.0,Manhattan,Chelsea and Clinton


In [33]:
nyc_irs.to_csv('./nyc_irs.csv', index=False)

In [32]:
x = pd.read_csv('./nyc_irs.csv', header=0)

x.head()

,Unnamed: 0,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,ELF,...,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000,Borough,Neighborhood
0,0,36,NY,10001,1,2960.0,2370.0,210.0,290.0,2770.0,...,420.0,426.0,2150.0,4855.0,2100.0,4666.0,50.0,159.0,Manhattan,Chelsea and Clinton
1,1,36,NY,10001,2,2470.0,1790.0,200.0,410.0,2370.0,...,580.0,1189.0,1860.0,5102.0,1820.0,4913.0,50.0,189.0,Manhattan,Chelsea and Clinton
2,2,36,NY,10001,3,1930.0,1530.0,180.0,160.0,1850.0,...,520.0,1376.0,1390.0,4045.0,1350.0,3859.0,50.0,186.0,Manhattan,Chelsea and Clinton
3,3,36,NY,10001,4,1560.0,1260.0,170.0,90.0,1500.0,...,540.0,1676.0,1000.0,2489.0,960.0,2114.0,50.0,375.0,Manhattan,Chelsea and Clinton
4,4,36,NY,10001,5,3270.0,2520.0,510.0,100.0,3140.0,...,1470.0,6680.0,1780.0,6655.0,1640.0,5050.0,170.0,1605.0,Manhattan,Chelsea and Clinton


In [ ]:
x = x.drop